In [1]:
# Install YOLOv8
!pip install ultralytics opencv-python-headless



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 53.5 MB/s eta 0:00:00


In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
import cv2
import os
from ultralytics import YOLO
from pathlib import Path

# ===============================
# Step 1: Path to your video in Drive
# ===============================
video_path = "/content/drive/MyDrive/video/data/finalcarla-2023-11-29_16.10.56.mp4"  # <-- change to your video path

# ===============================
# Step 2: Run YOLOv8 Detection
# ===============================
model = YOLO("yolov8n.pt")

results = model.predict(
    source=video_path,
    save=True,                 # save annotated video & images
    classes=[0, 2, 5, 7]       # restrict to person, car, bus, truck
)

print("✅ Detection finished! Only person, car, bus, and truck detected.")

# ===============================
# Step 3: Extract frames from detected video
# ===============================
output_dir = "detected_frames"
os.makedirs(output_dir, exist_ok=True)

# Get processed video path (YOLO saves inside runs/detect/predict by default)
detected_dir = Path(results[0].save_dir)
detected_video_path = list(detected_dir.glob("*.mp4"))[0]

cap = cv2.VideoCapture(str(detected_video_path))
frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_filename = os.path.join(output_dir, f"frame_{frame_count:05d}.jpg")
    cv2.imwrite(frame_filename, frame)
    frame_count += 1

cap.release()
print(f"✅ Saved {frame_count} detected frames in '{output_dir}'")

# ===============================
# Step 4: Copy results to Drive
# ===============================
!cp -r runs/detect/predict /content/drive/MyDrive/yolo_results_video
!cp -r detected_frames /content/drive/MyDrive/yolo_results_frames

print("✅ Results copied to Drive:")
print(" - Annotated video in: /content/drive/MyDrive/yolo_results_video")
print(" - Frames in: /content/drive/MyDrive/yolo_results_frames")


WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/1962) /content/drive/MyDrive/video/data/finalcarla-2023-11-29_16.10.56.mp4: 384x640 (no detections), 8.6ms
video 1/1 (frame 2/1962) /content/drive/MyDrive/video/data/finalcarla-2023-11-29_16.10.56.mp4: 384x640 (no detections), 8.7ms
video 1/1 (frame 3/1962) /content/drive/MyDrive/video/data/finalcarla-2023-11-29_16.10.56.mp4: 384x640 (no detections), 6.9ms
video 1/1 (frame 4/1962) /content/drive/MyDrive/video/data/finalcarla-2023-11-

In [4]:
!pip install ultralytics opencv-python-headless

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.8 MB/s eta 0:00:00


In [9]:
import os
import shutil
from ultralytics import YOLO
from google.colab import files

# ===============================
# Step 1: Define your image dataset folders
# ===============================
# Change these to your actual paths in Drive
dataset_paths = [
    "/content/drive/MyDrive/data_new/day",        # folder with images
    "/content/drive/MyDrive/data_new/day_rain",
    "/content/drive/MyDrive/data_new/night",
    "/content/drive/MyDrive/data_new/night_rain"
]

# ===============================
# Step 2: Setup YOLOv8 and output folder
# ===============================
# Where to store results in Drive
DRIVE_OUT = "/content/drive/MyDrive/YOLOv10_Image_Results"
os.makedirs(DRIVE_OUT, exist_ok=True)

# Restrict to these COCO classes: person=0, car=2, bus=5, truck=7
CLASSES = [0, 2, 5, 7]

# Choose YOLOv8 weight (n = nano, s = small, m = medium, l = large, x = xlarge)
MODEL_WEIGHTS = "yolov10n.pt"   # you can change to yolov8s.pt for more accuracy

# ===============================
# Load YOLOv8 model
# ===============================
model = YOLO(MODEL_WEIGHTS)

# ===============================
# Process each dataset
# ===============================
for ds in dataset_paths:
    dataset_name = os.path.basename(ds)  # use actual folder name (day, night, etc.)
    dataset_out_dir = os.path.join(DRIVE_OUT, dataset_name)
    input_dir = os.path.join(dataset_out_dir, "input_images")
    output_dir = os.path.join(dataset_out_dir, "output_images")

    print(f"\n🔹 Processing {dataset_name}: {ds}")
    os.makedirs(input_dir, exist_ok=True)
    os.makedirs(output_dir, exist_ok=True)

    # A) Copy input images
    shutil.copytree(ds, input_dir, dirs_exist_ok=True)
    print(f"✅ Copied input images → {input_dir}")

    # B) Run YOLOv8 detection
    model.predict(
        source=ds,
        classes=CLASSES,
        save=True,
        project=dataset_out_dir,   # save results inside this dataset’s folder
        name="output_images",
        exist_ok=True
    )
    print(f"✅ Saved annotated images → {output_dir}")

    # C) Zip the whole dataset folder (input + output)
    zip_path = f"{dataset_out_dir}.zip"
    shutil.make_archive(dataset_out_dir, 'zip', dataset_out_dir)
    print(f"🗜️ Zipped results → {zip_path}")

    # D) Auto-download the zip
    files.download(zip_path)

print("\n🎉 All datasets processed! Check zipped results in your Drive at:")
print(DRIVE_OUT)



🔹 Processing day: /content/drive/MyDrive/data_new/day
✅ Copied input images → /content/drive/MyDrive/YOLOv10_Image_Results/day/input_images

image 1/1 /content/drive/MyDrive/data_new/day/frame_00028.jpg: 384x640 1 person, 4 cars, 14.7ms
Speed: 3.0ms preprocess, 14.7ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)
Results saved to /content/drive/MyDrive/YOLOv10_Image_Results/day/output_images
✅ Saved annotated images → /content/drive/MyDrive/YOLOv10_Image_Results/day/output_images
🗜️ Zipped results → /content/drive/MyDrive/YOLOv10_Image_Results/day.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


🔹 Processing day_rain: /content/drive/MyDrive/data_new/day_rain
✅ Copied input images → /content/drive/MyDrive/YOLOv10_Image_Results/day_rain/input_images

image 1/1 /content/drive/MyDrive/data_new/day_rain/frame_00085.jpg: 352x640 1 car, 15.6ms
Speed: 3.0ms preprocess, 15.6ms inference, 1.0ms postprocess per image at shape (1, 3, 352, 640)
Results saved to /content/drive/MyDrive/YOLOv10_Image_Results/day_rain/output_images
✅ Saved annotated images → /content/drive/MyDrive/YOLOv10_Image_Results/day_rain/output_images
🗜️ Zipped results → /content/drive/MyDrive/YOLOv10_Image_Results/day_rain.zip


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>


🔹 Processing night_rain: /content/drive/MyDrive/data_new/night_rain
✅ Copied input images → /content/drive/MyDrive/YOLOv10_Image_Results/night_rain/input_images

image 1/1 /content/drive/MyDrive/data_new/night_rain/frame_00349.jpg: 352x640 1 truck, 8.8ms
Speed: 1.9ms preprocess, 8.8ms inference, 0.7ms postprocess per image at shape (1, 3, 352, 640)
Results saved to /content/drive/MyDrive/YOLOv10_Image_Results/night_rain/output_images
✅ Saved annotated images → /content/drive/MyDrive/YOLOv10_Image_Results/night_rain/output_images
🗜️ Zipped results → /content/drive/MyDrive/YOLOv10_Image_Results/night_rain.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


🎉 All datasets processed! Check zipped results in your Drive at:
/content/drive/MyDrive/YOLOv10_Image_Results


In [11]:
import os
import shutil
from ultralytics import YOLO
from google.colab import files

# ===============================
# Step 1: Define your image dataset folders
# ===============================
# Change these to your actual paths in Drive
dataset_paths = [
  "/content/drive/MyDrive/data_new/day",        # folder with images
    "/content/drive/MyDrive/data_new/day_rain",
    "/content/drive/MyDrive/data_new/night",
    "/content/drive/MyDrive/data_new/night_rain"
]

# ===============================
# Step 2: Setup YOLOv8 and output folder
# ===============================
# Where to store results in Drive
DRIVE_OUT = "/content/drive/MyDrive/YOLOv8_Image_Results"
os.makedirs(DRIVE_OUT, exist_ok=True)

# Restrict to these COCO classes: person=0, car=2, bus=5, truck=7
CLASSES = [0, 2, 5, 7]

# Choose YOLOv8 weight (n = nano, s = small, m = medium, l = large, x = xlarge)
MODEL_WEIGHTS = "yolov8n.pt"   # you can change to yolov8s.pt for more accuracy

# ===============================
# Load YOLOv8 model
# ===============================
model = YOLO(MODEL_WEIGHTS)

# ===============================
# Process each dataset
# ===============================
for ds in dataset_paths:
    dataset_name = os.path.basename(ds)  # use actual folder name (day, night, etc.)
    dataset_out_dir = os.path.join(DRIVE_OUT, dataset_name)
    input_dir = os.path.join(dataset_out_dir, "input_images")
    output_dir = os.path.join(dataset_out_dir, "output_images")

    print(f"\n🔹 Processing {dataset_name}: {ds}")
    os.makedirs(input_dir, exist_ok=True)
    os.makedirs(output_dir, exist_ok=True)

    # A) Copy input images
    shutil.copytree(ds, input_dir, dirs_exist_ok=True)
    print(f"✅ Copied input images → {input_dir}")

    # B) Run YOLOv8 detection
    model.predict(
        source=ds,
        classes=CLASSES,
        save=True,
        project=dataset_out_dir,   # save results inside this dataset’s folder
        name="output_images",
        exist_ok=True
    )
    print(f"✅ Saved annotated images → {output_dir}")

    # C) Zip the whole dataset folder (input + output)
    zip_path = f"{dataset_out_dir}.zip"
    shutil.make_archive(dataset_out_dir, 'zip', dataset_out_dir)
    print(f"🗜️ Zipped results → {zip_path}")

    # D) Auto-download the zip
    files.download(zip_path)

print("\n🎉 All datasets processed! Check zipped results in your Drive at:")
print(DRIVE_OUT)



🔹 Processing day: /content/drive/MyDrive/data_new/day
✅ Copied input images → /content/drive/MyDrive/YOLOv8_Image_Results/day/input_images

image 1/1 /content/drive/MyDrive/data_new/day/frame_00028.jpg: 384x640 1 person, 3 cars, 6.7ms
Speed: 1.9ms preprocess, 6.7ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)
Results saved to /content/drive/MyDrive/YOLOv8_Image_Results/day/output_images
✅ Saved annotated images → /content/drive/MyDrive/YOLOv8_Image_Results/day/output_images
🗜️ Zipped results → /content/drive/MyDrive/YOLOv8_Image_Results/day.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


🔹 Processing day_rain: /content/drive/MyDrive/data_new/day_rain
✅ Copied input images → /content/drive/MyDrive/YOLOv8_Image_Results/day_rain/input_images

image 1/1 /content/drive/MyDrive/data_new/day_rain/frame_00085.jpg: 352x640 2 persons, 3 cars, 7.0ms
Speed: 1.9ms preprocess, 7.0ms inference, 1.6ms postprocess per image at shape (1, 3, 352, 640)
Results saved to /content/drive/MyDrive/YOLOv8_Image_Results/day_rain/output_images
✅ Saved annotated images → /content/drive/MyDrive/YOLOv8_Image_Results/day_rain/output_images
🗜️ Zipped results → /content/drive/MyDrive/YOLOv8_Image_Results/day_rain.zip


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>


🔹 Processing night_rain: /content/drive/MyDrive/data_new/night_rain
✅ Copied input images → /content/drive/MyDrive/YOLOv8_Image_Results/night_rain/input_images

image 1/1 /content/drive/MyDrive/data_new/night_rain/frame_00349.jpg: 352x640 2 cars, 1 truck, 11.7ms
Speed: 2.9ms preprocess, 11.7ms inference, 2.2ms postprocess per image at shape (1, 3, 352, 640)
Results saved to /content/drive/MyDrive/YOLOv8_Image_Results/night_rain/output_images
✅ Saved annotated images → /content/drive/MyDrive/YOLOv8_Image_Results/night_rain/output_images
🗜️ Zipped results → /content/drive/MyDrive/YOLOv8_Image_Results/night_rain.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


🎉 All datasets processed! Check zipped results in your Drive at:
/content/drive/MyDrive/YOLOv8_Image_Results


In [ ]:
import os
import shutil
from ultralytics import YOLO
from google.colab import files

# ===============================
# Step 1: Define your image dataset folders
# ===============================
# Change these to your actual paths in Drive
dataset_paths = [
    "/content/drive/MyDrive/data/day",        # folder with images
    "/content/drive/MyDrive/data/day_rain",
    "/content/drive/MyDrive/data/night",
    "/content/drive/MyDrive/data/night_rain"
]

# ===============================
# Step 2: Setup YOLO and output folder
# ===============================
# Where to store results in Drive
DRIVE_OUT = "/content/drive/MyDrive/YOLOv9_Image_Results"
os.makedirs(DRIVE_OUT, exist_ok=True)

# Restrict to these COCO classes: person=0, car=2, bus=5, truck=7
CLASSES = [0, 2, 5, 7]

# Choose YOLOv9 weight (c = balanced, e = larger)
MODEL_WEIGHTS = "yolov9c.pt"

# ===============================
# Load YOLOv9 model
# ===============================
model = YOLO(MODEL_WEIGHTS)

# ===============================
# Process each dataset
# ===============================
for i, ds in enumerate(dataset_paths, start=1):
    dataset_name = f"dataset{i}"
    dataset_out_dir = os.path.join(DRIVE_OUT, dataset_name)
    input_dir = os.path.join(dataset_out_dir, "input_images")
    output_dir = os.path.join(dataset_out_dir, "output_images")

    print(f"\n🔹 Processing {dataset_name}: {ds}")
    os.makedirs(input_dir, exist_ok=True)
    os.makedirs(output_dir, exist_ok=True)

    # A) Copy input images
    shutil.copytree(ds, input_dir, dirs_exist_ok=True)
    print(f"✅ Copied input images → {input_dir}")

    # B) Run YOLOv9 detection
    model.predict(
        source=ds,
        classes=CLASSES,
        save=True,
        project=dataset_out_dir,   # save results inside this dataset’s folder
        name="output_images",
        exist_ok=True
    )
    print(f"✅ Saved annotated images → {output_dir}")

    # C) Zip the whole dataset folder (input + output)
    zip_path = f"{dataset_out_dir}.zip"
    shutil.make_archive(dataset_out_dir, 'zip', dataset_out_dir)
    print(f"🗜️ Zipped results → {zip_path}")

    # D) Auto-download the zip
    files.download(zip_path)

print("\n🎉 All datasets processed! Check zipped results in your Drive at:")
print(DRIVE_OUT)
